In [1]:
foldr :: (a->b->b)->b->[a]->b
foldr f z [] = z
foldr f z (x:xs) = f x (foldr f z xs)

Line 2: Use foldr
Found:
foldr f z [] = z
foldr f z (x : xs) = f x (foldr f z xs)
Why not:
foldr f z xs = foldr f z xs

In [2]:
foldr (+) 0 [1,2,3,4,5]
-- foldr ++ [] [1,2,3,4,5]

Line 1: Use sum
Found:
foldr (+) 0
Why not:
sum

15

In [3]:
xs = map show [1..5]
-- foldr' (\x y -> concat ["(",x,"+",y,")"]) "0" xs

In [4]:
myAny :: (a->Bool)->[a]->Bool
myAny f xs = foldr (\x b -> f x || b) False xs
myAny even [1..]

Line 2: Eta reduce
Found:
myAny f xs = foldr (\ x b -> f x || b) False xs
Why not:
myAny f = foldr (\ x b -> f x || b) False

True

In [5]:
length $ take 2 $ take 4 ([1, 2]++undefined)

2

In [6]:
foldl :: (b->a->b)->b->[a]->b
foldl f acc [] = acc
foldl f acc (x:xs) = foldl f (f acc x) xs
-- foldl (+) 0 [1,2,3,4,5]
f = \x y -> concat ["(",x,"+",y,")"]
foldr f "0" (map show [1..5])
foldl f "0" (map show [1..5])

Line 5: Redundant lambda
Found:
f = \ x y -> concat ["(", x, "+", y, ")"]
Why not:
f x y = concat ["(", x, "+", y, ")"]Line 6: Fuse foldr/map
Found:
foldr f "0" (map show [1 .. 5])
Why not:
foldr (f . show) "0" [1 .. 5]

"(1+(2+(3+(4+(5+0)))))"

"(((((0+1)+2)+3)+4)+5)"

In [7]:
foldr (+) 0 [1..5]
scanr (+) 0 [1..5]

Line 1: Use sum
Found:
foldr (+) 0
Why not:
sum

15

[15,14,12,9,5,0]

In [8]:
foldl (+) 0 [1..5]
scanl (+) 0 [1..5]

Line 1: Use sum
Found:
foldl (+) 0
Why not:
sum

15

[0,1,3,6,10,15]

In [9]:
foldr (++) [] ["woot","WOOT","woot"]
foldr max 'a' "fear is the little death"
foldr (&&) True [False,True]
foldr (||) True [False,True]

Line 1: Use concat
Found:
foldr (++) []
Why not:
concatLine 3: Use and
Found:
foldr (&&) True
Why not:
and

"wootWOOTwoot"

't'

False

True

In [10]:
foldl (flip ((++) . show)) "" [1..5]
foldr (flip const) 'a' [1..5]
foldl const 0 "tacos"
foldl (flip const) 'b' "burritos"
foldl const 'z' [1..5]

Line 2: Redundant flip
Found:
flip const
Why not:
\ _ x -> xLine 4: Redundant flip
Found:
flip const
Why not:
\ _ x -> x

"54321"

'a'

0

's'

'z'

In [11]:
:t foldr (flip const)

foldr (flip const) :: forall {b} {a}. b -> [a] -> b

In [12]:

-- foldl const 0 [1,2,3,undefined]
-- foldr const 0 ([1..5] ++ undefined ++ undefined)
-- scanr const 0 ([1..5] ++ undefined ++ undefined)
foldr const 0 [1,2,3,4,5]

1

In [13]:
foldr (\a b -> (take 3) a ++ b ) "" ["Pizza","Apple","Banana"]

Line 1: Redundant bracket
Found:
(take 3) a
Why not:
take 3 a

"PizAppBan"

## Database Processing

In [26]:
import Data.Time
data DatabaseItem = DbString String
                | DbNumber Integer
                | DbDate UTCTime
                deriving (Eq,Ord,Show)


In [28]:
theDatabase :: [DatabaseItem]
theDatabase = 
    [   DbDate (UTCTime  (fromGregorian 1911 5 1)  (secondsToDiffTime 34123))  ,
        DbNumber 9001  ,
        DbString "Hello, world!"  ,
        DbDate (UTCTime  (fromGregorian 1921 5 1)  (secondsToDiffTime 34123))  ]

In [ ]:
:t foldr

foldr :: forall a b. (a -> b -> b) -> b -> [a] -> b

In [ ]:
fdate :: DatabaseItem -> UTCTime

In [29]:
filterDbDate :: [DatabaseItem] -> [UTCTime]
filterDbDate = foldr f []
    where f x res = 
            case x of 
                (DbDate utc) -> utc:res
                _ -> res


-- filterDbDate item= map g (filter f item)
--     where f (DbDate _) = True
--           f  _ = False
--           g (DbDate utc) = utc
filterDbDate theDatabase

[1911-05-01 09:28:43 UTC,1921-05-01 09:28:43 UTC]

In [30]:
filterDbNumber :: [DatabaseItem]-> [Integer]
filterDbNumber = foldr f []
    where f x acc = 
            case x of 
                (DbNumber int)->int:acc
                _->acc
filterDbNumber theDatabase

[9001]

In [31]:
mostRecent :: [DatabaseItem] -> UTCTime
mostRecent = foldr f (UTCTime  (fromGregorian 0000 1 1)  (secondsToDiffTime 34123))
    where 
        f (DbDate t) max_time = max t max_time
        f _ max_time = max_time
mostRecent theDatabase
          

1921-05-01 09:28:43 UTC

In [46]:
sumDb :: [DatabaseItem] -> Integer
sumDb = foldr f 0 
        where f x acc = case x of
                DbNumber n -> acc + n
                _ -> acc
sumDb theDatabase

9001

In [50]:
avgDb :: [DatabaseItem]->Double
avgDb db = (fromIntegral (sumDb db)) / (fromIntegral (countDb db) )
    where 
        countDb db = length (filter f db)
        f (DbNumber _) = True
        f _ = False
avgDb theDatabase

Line 2: Redundant bracket
Found:
(fromIntegral (sumDb db)) / (fromIntegral (countDb db))
Why not:
fromIntegral (sumDb db) / (fromIntegral (countDb db))Line 2: Redundant bracket
Found:
(fromIntegral (sumDb db)) / (fromIntegral (countDb db))
Why not:
(fromIntegral (sumDb db)) / fromIntegral (countDb db)

9001.0

In [ ]:
take 3 $ foldl (flip(:)) [] [1..] 

In [2]:
fibs = 1 : scanl (+) 1 fibs
takeWhile (< 100) fibs 

[1,1,2,3,5,8,13,21,34,55,89]

In [9]:
factorial x=take x  (scanl (*)  1 [2..])
factorial 1

[1]

In [37]:
-- fst' (_,_,x)=x
f :: (Char,b,c)-> Bool
f (x,_,_) = x == 'p'
f _ = False
f' tup = 
    case tup of
        (x,_,_) -> x == 'p'
        _ -> False

## Chapter Exercises

In [ ]:
stops = "pbtdkg"
vowels = "aeiou"
-- filter (\x -> case x of
--             (_,_,x) -> x == 'p'
--             _ -> False
--              ) [(x,y,z) | x<-stops,y<-vowels,z<-stops]
filter f' [(x,y,z) | x<-stops,y<-vowels,z<-stops]


[('p','a','p'),('p','a','b'),('p','a','t'),('p','a','d'),('p','a','k'),('p','a','g'),('p','e','p'),('p','e','b'),('p','e','t'),('p','e','d'),('p','e','k'),('p','e','g'),('p','i','p'),('p','i','b'),('p','i','t'),('p','i','d'),('p','i','k'),('p','i','g'),('p','o','p'),('p','o','b'),('p','o','t'),('p','o','d'),('p','o','k'),('p','o','g'),('p','u','p'),('p','u','b'),('p','u','t'),('p','u','d'),('p','u','k'),('p','u','g')]

In [42]:
nouns=["apple","chicken","banana","bear","tiger"]
verbs = ["eat","drink","run"]
[x++"-"++y++"-"++z | x<-nouns,y<-verbs,z<-nouns]

["apple-eat-apple","apple-eat-chicken","apple-eat-banana","apple-eat-bear","apple-eat-tiger","apple-drink-apple","apple-drink-chicken","apple-drink-banana","apple-drink-bear","apple-drink-tiger","apple-run-apple","apple-run-chicken","apple-run-banana","apple-run-bear","apple-run-tiger","chicken-eat-apple","chicken-eat-chicken","chicken-eat-banana","chicken-eat-bear","chicken-eat-tiger","chicken-drink-apple","chicken-drink-chicken","chicken-drink-banana","chicken-drink-bear","chicken-drink-tiger","chicken-run-apple","chicken-run-chicken","chicken-run-banana","chicken-run-bear","chicken-run-tiger","banana-eat-apple","banana-eat-chicken","banana-eat-banana","banana-eat-bear","banana-eat-tiger","banana-drink-apple","banana-drink-chicken","banana-drink-banana","banana-drink-bear","banana-drink-tiger","banana-run-apple","banana-run-chicken","banana-run-banana","banana-run-bear","banana-run-tiger","bear-eat-apple","bear-eat-chicken","bear-eat-banana","bear-eat-bear","bear-eat-tiger","bear-dri

In [53]:
seekritFunc x =  fromIntegral (sum (map length (words x))) /  fromIntegral (length (words x))
seekritFunc "Hello Worldcc xxxc"

-- words "Hexxx cxcc cc"

5.333333333333333

In [55]:
myOr :: [Bool]->Bool
myOr = foldr (||) False
myOr [False,True]

Line 2: Use or
Found:
foldr (||) False
Why not:
or

True

In [60]:
myAny :: (a->Bool)->[a]->Bool
myAny f = foldr (\x y -> f x || y) False
myAny odd [1,3,5]

True

In [71]:
myElem :: Eq a => a->[a]->Bool
myElem e = foldr (\x y -> x==e ||y) False
    -- where f x y
    --         | x == e = True
    --         | otherwise = y
-- myElem 5 [2..10]
-- :t any
myElem' e = any (==e) 
myElem' 5 [2..10]

Line 8: Use elem
Found:
any (== e)
Why not:
elem e

True

In [73]:
myReverse :: [a]->[a]
myReverse  = foldl (flip (:)) []
myReverse [1,2,3,4]

[4,3,2,1]

In [74]:
myMap :: (a->b)->[a]->[b]
myMap f = foldr (\x y-> (f x): y) []
    -- where g x y = (f x):y
myMap (+1) [1,2,3,4,5]

Line 2: Use map
Found:
foldr (\ x y -> (f x) : y) []
Why not:
map (\ x -> f x)Line 2: Redundant bracket
Found:
(f x) : y
Why not:
f x : y

[2,3,4,5,6]

In [82]:
myFilter :: (a->Bool) -> [a] -> [a]
myFilter f = foldr (\x y-> if f x then x:y else y) []
    -- where g x y = 
    --         case f x of
    --             True -> x:y
    --             _ -> y
myFilter (> 3) [1..5]

[4,5]

In [84]:
squish :: [[a]]->[a]
squish = foldr (\x ys-> x ++ ys) []
squish [[1,2,3],[4,5,6]]

Line 2: Avoid lambda
Found:
\ x ys -> x ++ ys
Why not:
(++)

[1,2,3,4,5,6]

In [87]:
squishMap :: (a->[b])->[a]->[b]
squishMap f = foldr (\x ys -> f x ++ ys) []
squishMap (\x -> "WO " ++ [x] ++ " OT ") "blah"

"WO b OT WO l OT WO a OT WO h OT "

In [ ]:
squishAgain = squishMap id
squishAgain [[1,2,3],[4,5,6]]

In [38]:
myMaximumBy :: (a->a->Ordering)->[a]->a
myMaximumBy f l= foldl g (head l) l
    where g x acc = 
            case f acc x of
                GT -> acc
                _  -> x
myMaximumBy (\_ _ -> LT) [1..10]

1

In [43]:
myMinimumBy :: (a->a->Ordering)->[a]->a
myMinimumBy f l= foldr g (head l) l
    where g x y = 
            case f y x of
                GT -> x
                _  -> y
-- myMinimumBy compare [2..10]
myMinimumBy' f l=  myMaximumBy (flip f) l
myMinimumBy' (\_ _ -> GT) [1..10]
myMinimumBy compare [1..10]

Line 8: Eta reduce
Found:
myMinimumBy' f l = myMaximumBy (flip f) l
Why not:
myMinimumBy' f = myMaximumBy (flip f)

10

1